In [1]:
# Query for all SmartSPIM assets to find unique subjects
from aind_data_access_api.document_db import MetadataDbClient

# Load Excel spreadsheet data
import sys
import pandas as pd
sys.path.append('/Users/doug.ollerenshaw/code/aind-workbench/projects/patchseq_metadata')

from excel_loader import load_specimen_procedures_excel

In [2]:
# Initialize the database client
client = MetadataDbClient(
    host="api.allenneuraldynamics.org",
    database="metadata_index", 
    collection="data_assets"
)

# Query for all records with "SmartSPIM" in the name
smartspim_records = client.retrieve_docdb_records(
    filter_query={"name": {"$regex": "SmartSPIM", "$options": "i"}},
    projection={"subject.subject_id": 1, "name": 1}
)

print(f"Total SmartSPIM records found: {len(smartspim_records)}")

# Extract unique subject IDs
unique_subjects = set()
for record in smartspim_records:
    if 'subject' in record and 'subject_id' in record['subject']:
        unique_subjects.add(record['subject']['subject_id'])

print(f"Number of unique subjects with SmartSPIM assets: {len(unique_subjects)}")
print(f"Unique subject IDs: {sorted(list(unique_subjects))}")

Total SmartSPIM records found: 5521
Number of unique subjects with SmartSPIM assets: 2491
Unique subject IDs: ['156631', '18559', '19603', '3283906', '3283909', '393', '394', '397', '568105', '595922', '597305', '603261', '605078', '605079', '607463', '608846', '610879', '611328', '611334', '612962', '613813', '614952', '614956', '619778', '620326', '620633', '620642', '620643', '620796', '621362', '622280', '622386', '622464', '622466', '622468', '622638', '622640', '622957', '623705', '623707', '623710', '623711', '623894', '623900', '624643', '624644', '624645', '624646', '625098', '625099', '625100', '625382', '625463', '625464', '625749', '626182', '626185', '627137', '628286', '628289', '628290', '631680', '631709', '631710', '631977', '631978', '632268', '632521', '633190', '634568', '634569', '634571', '638312', '638314', '638315', '638316', '638665', '638779', '638781', '638927', '638929', '639090', '639091', '639092', '639374', '639378', '639485', '640387', '640390', '640391'

In [3]:
# Load the spreadsheet data
print("Loading Excel spreadsheet data...")
sheet_data = load_specimen_procedures_excel()

# Extract unique subject IDs from the Batch Info sheet
spreadsheet_subjects = set()
if sheet_data and "Batch Info" in sheet_data:
    batch_info = sheet_data["Batch Info"]
    for idx, row in batch_info.iterrows():
        lab_track_ids = str(row['LabTrack IDs']) if pd.notna(row['LabTrack IDs']) else ""
        # Split by comma and clean up whitespace
        if lab_track_ids and lab_track_ids != 'nan':
            # Split by comma, strip whitespace, and filter out empty strings
            subjects = [s.strip() for s in lab_track_ids.split(',') if s.strip()]
            for subject in subjects:
                # Only add if it's all digits (valid subject ID)
                if subject.isdigit():
                    spreadsheet_subjects.add(subject)

print(f"Found {len(spreadsheet_subjects)} unique subjects in spreadsheet")
print(f"Sample subjects: {sorted(list(spreadsheet_subjects))[:10]}")

Loading Excel spreadsheet data...
Found 1504 unique subjects in spreadsheet
Sample subjects: ['018559', '019603', '3283906', '3283909', '46044', '568103', '602630', '607398', '608846', '611328']


In [4]:
# Get all SmartSPIM records with the fields we need
print("Querying database for all SmartSPIM records...")

smartspim_with_procedures = client.retrieve_docdb_records(
    filter_query={"name": {"$regex": "SmartSPIM", "$options": "i"}},
    projection={
        "name": 1, 
        "subject.subject_id": 1, 
        "procedures": 1,  # Get the full procedures object
        "data_description.project_name": 1,
        "data_description.institution.abbreviation": 1,
    }
)

print(f"Retrieved {len(smartspim_with_procedures)} SmartSPIM records")

Querying database for all SmartSPIM records...
Retrieved 5521 SmartSPIM records


In [5]:
# Process each record and build out a summary DataFrame
data = []
for record in smartspim_with_procedures:
    # Extract subject ID
    subject_id = record.get('subject', {}).get('subject_id')
    if not subject_id:
        continue
    
    # Extract asset name
    asset_name = record.get('name', 'Unknown')
    
    # Extract collection date from asset name
    collection_date = "Unknown"
    try:
        parts = asset_name.split('_')
        for part in parts:
            if len(part) == 10 and part.count('-') == 2:  # YYYY-MM-DD format
                collection_date = part
                break
    except:
        pass
    
    # Check if procedures exists and has content
    procedures = record.get('procedures', {})
    has_procedures = bool(procedures and len(procedures) > 0)
    
    # Check specimen_procedures specifically
    has_specimen_procedures = False
    if procedures:
        specimen_procedures = procedures.get('specimen_procedures', [])
        has_specimen_procedures = len(specimen_procedures) > 0
    
    # Check if mouse is in spreadsheet
    in_spreadsheet = str(subject_id) in spreadsheet_subjects
    
    data.append({
        'subject_id': subject_id,
        'collection_date': collection_date,
        'has_procedures': has_procedures,
        'has_specimen_procedures': has_specimen_procedures,
        'asset_name': asset_name,
        'project_name': record.get('data_description', {}).get('project_name', 'Unknown'),
        'institution': record.get('data_description', {}).get('institution', {}).get('abbreviation', 'Unknown'),
        'in_spreadsheet': in_spreadsheet
    })

# Create DataFrame
summary_df = pd.DataFrame(data)

# Sort by collection date
summary_df = summary_df.sort_values('collection_date').reset_index(drop=True)

summary_df


,subject_id,collection_date,has_procedures,has_specimen_procedures,asset_name,project_name,institution,in_spreadsheet
0,622640,2022-05-31,False,False,SmartSPIM_622640_2022-05-31_12-01-30_stitched_...,None,Unknown,False
1,622640,2022-05-31,False,False,SmartSPIM_622640_2022-05-31_12-01-30,None,Unknown,False
2,620796,2022-05-31,False,False,SmartSPIM_620796_2022-05-31_17-54-53,None,Unknown,False
3,620796,2022-05-31,False,False,SmartSPIM_620796_2022-05-31_17-54-53_stitched_...,None,Unknown,False
4,622957,2022-06-07,False,False,SmartSPIM_622957_2022-06-07_10-54-57_stitched_...,None,Unknown,False
...,...,...,...,...,...,...,...,...
5404,791715,2025-08-19,True,True,SmartSPIM_791715_2025-08-19_11-19-07,Genetic Perturbation Platform,AIND,True
5405,801624,2025-08-19,True,True,SmartSPIM_801624_2025-08-19_19-51-26,Discovery-Neuromodulator circuit dynamics duri...,AIND,True
5406,804714,2025-08-19,True,True,SmartSPIM_804714_2025-08-19_23-07-42,Discovery-Neuromodulator circuit dynamics duri...,AIND,True
5407,810235,2025-08-19,True,True,SmartSPIM_810235_2025-08-19_17-27-19,Thalamus in the middle - Project 1 Mesoscale t...,AIND,True


In [6]:
summary_df['has_procedures'].value_counts()

has_procedures
True     3591
False    1818
Name: count, dtype: int64

In [7]:
summary_df['has_specimen_procedures'].value_counts()

has_specimen_procedures
False    3469
True     1940
Name: count, dtype: int64

In [8]:
summary_df.query('has_specimen_procedures == True').tail(25)

,subject_id,collection_date,has_procedures,has_specimen_procedures,asset_name,project_name,institution,in_spreadsheet
5363,796011,2025-08-12,True,True,SmartSPIM_796011_2025-08-12_14-29-14,Thalamus in the middle - Project 1 Mesoscale t...,AIND,True
5378,809093,2025-08-14,True,True,SmartSPIM_809093_2025-08-14_18-33-13,Thalamus in the middle - Project 1 Mesoscale t...,AIBS,True
5384,809093,2025-08-14,True,True,SmartSPIM_809093_2025-08-14_18-33-13_stitched_...,Thalamus in the middle - Project 1 Mesoscale t...,AIBS,True
5385,770103,2025-08-15,True,True,SmartSPIM_770103_2025-08-15_10-17-43,Discovery-Neuromodulator circuit dynamics duri...,AIND,True
5386,782229,2025-08-15,True,True,SmartSPIM_782229_2025-08-15_15-03-33,Genetic Perturbation Platform,AIND,True
5387,805163,2025-08-15,True,True,SmartSPIM_805163_2025-08-15_12-20-35,MSMA Platform,AIND,True
5388,770103,2025-08-15,True,True,SmartSPIM_770103_2025-08-15_10-17-43_stitched_...,Discovery-Neuromodulator circuit dynamics duri...,AIND,True
5389,784635,2025-08-15,True,True,SmartSPIM_784635_2025-08-15_19-39-47,Genetic Perturbation Platform,AIND,True
5390,784635,2025-08-15,True,True,SmartSPIM_784635_2025-08-15_19-39-47_stitched_...,Genetic Perturbation Platform,AIND,True
5391,782229,2025-08-15,True,True,SmartSPIM_782229_2025-08-15_15-03-33_stitched_...,Genetic Perturbation Platform,AIND,True


In [9]:
summary_df.query('in_spreadsheet == True').tail(25)

,subject_id,collection_date,has_procedures,has_specimen_procedures,asset_name,project_name,institution,in_spreadsheet
5384,809093,2025-08-14,True,True,SmartSPIM_809093_2025-08-14_18-33-13_stitched_...,Thalamus in the middle - Project 1 Mesoscale t...,AIBS,True
5385,770103,2025-08-15,True,True,SmartSPIM_770103_2025-08-15_10-17-43,Discovery-Neuromodulator circuit dynamics duri...,AIND,True
5386,782229,2025-08-15,True,True,SmartSPIM_782229_2025-08-15_15-03-33,Genetic Perturbation Platform,AIND,True
5387,805163,2025-08-15,True,True,SmartSPIM_805163_2025-08-15_12-20-35,MSMA Platform,AIND,True
5388,770103,2025-08-15,True,True,SmartSPIM_770103_2025-08-15_10-17-43_stitched_...,Discovery-Neuromodulator circuit dynamics duri...,AIND,True
5389,784635,2025-08-15,True,True,SmartSPIM_784635_2025-08-15_19-39-47,Genetic Perturbation Platform,AIND,True
5390,784635,2025-08-15,True,True,SmartSPIM_784635_2025-08-15_19-39-47_stitched_...,Genetic Perturbation Platform,AIND,True
5391,782229,2025-08-15,True,True,SmartSPIM_782229_2025-08-15_15-03-33_stitched_...,Genetic Perturbation Platform,AIND,True
5392,779209,2025-08-18,True,True,SmartSPIM_779209_2025-08-18_20-45-19_stitched_...,Discovery-Neuromodulator circuit dynamics duri...,AIND,True
5393,768517,2025-08-18,True,True,SmartSPIM_768517_2025-08-18_16-53-35_stitched_...,Discovery-Neuromodulator circuit dynamics duri...,AIND,True


In [10]:
summary_df.query('has_specimen_procedures == False').tail(25)

,subject_id,collection_date,has_procedures,has_specimen_procedures,asset_name,project_name,institution,in_spreadsheet
5345,769362,2025-08-11,True,False,SmartSPIM_769362_2025-08-11_21-36-34,Genetic Perturbation Platform,AIND,True
5346,769359,2025-08-11,True,False,SmartSPIM_769359_2025-08-11_17-20-33,Genetic Perturbation Platform,AIND,True
5347,767538,2025-08-11,True,False,SmartSPIM_767538_2025-08-11_11-38-03,Genetic Perturbation Platform,AIND,True
5348,765365,2025-08-11,True,False,SmartSPIM_765365_2025-08-11_12-25-10,Genetic Perturbation Platform,AIND,True
5364,807692,2025-08-13,True,False,SmartSPIM_807692_2025-08-13_19-03-35,CTY Genetic Tools,AIBS,False
5365,807691,2025-08-13,True,False,SmartSPIM_807691_2025-08-13_14-54-59,CTY Genetic Tools,AIBS,False
5366,807693,2025-08-13,True,False,SmartSPIM_807693_2025-08-13_15-41-02_stitched_...,CTY Genetic Tools,AIBS,False
5367,807691,2025-08-13,True,False,SmartSPIM_807691_2025-08-13_14-54-59_stitched_...,CTY Genetic Tools,AIBS,False
5368,807692,2025-08-13,True,False,SmartSPIM_807692_2025-08-13_19-03-35_stitched_...,CTY Genetic Tools,AIBS,False
5369,805461,2025-08-13,True,False,SmartSPIM_805461_2025-08-13_21-25-35,CTY Genetic Tools,AIBS,False


In [11]:
summary_df.query('has_specimen_procedures == False and institution == "AIND" and in_spreadsheet == True').tail(20)

,subject_id,collection_date,has_procedures,has_specimen_procedures,asset_name,project_name,institution,in_spreadsheet
5328,769642,2025-08-08,True,False,SmartSPIM_769642_2025-08-08_15-26-05_stitched_...,Genetic Perturbation Platform,AIND,True
5329,767463,2025-08-08,True,False,SmartSPIM_767463_2025-08-08_10-57-54_stitched_...,Genetic Perturbation Platform,AIND,True
5330,767463,2025-08-08,True,False,SmartSPIM_767463_2025-08-08_10-57-54,Genetic Perturbation Platform,AIND,True
5332,769642,2025-08-08,True,False,SmartSPIM_769642_2025-08-08_15-26-05,Genetic Perturbation Platform,AIND,True
5333,754430,2025-08-09,True,False,SmartSPIM_754430_2025-08-09_00-07-11_stitched_...,Discovery-Neuromodulator circuit dynamics duri...,AIND,True
5334,754430,2025-08-09,True,False,SmartSPIM_754430_2025-08-09_00-07-11,Discovery-Neuromodulator circuit dynamics duri...,AIND,True
5335,769359,2025-08-11,True,False,SmartSPIM_769359_2025-08-11_17-20-33_stitched_...,Genetic Perturbation Platform,AIND,True
5336,769362,2025-08-11,True,False,SmartSPIM_769362_2025-08-11_21-36-34_stitched_...,Genetic Perturbation Platform,AIND,True
5337,767538,2025-08-11,True,False,SmartSPIM_767538_2025-08-11_11-38-03_stitched_...,Genetic Perturbation Platform,AIND,True
5338,765365,2025-08-11,True,False,SmartSPIM_765365_2025-08-11_12-25-10_stitched_...,Genetic Perturbation Platform,AIND,True


In [23]:
import plotly.express as px
import pandas as pd
import numpy as np
from datetime import datetime

# Convert collection_date to datetime for better plotting
df_plot = summary_df.copy()
df_plot = df_plot[df_plot['collection_date'] != 'Unknown']  # Remove unknown dates
df_plot['collection_date_dt'] = pd.to_datetime(df_plot['collection_date'])

# Sort by date
df_plot = df_plot.sort_values('collection_date_dt')

# Convert boolean to string for better hover display
df_plot['procedures_status'] = df_plot['has_specimen_procedures'].map({True: 'Has Procedures', False: 'No Procedures'})
df_plot['spreadsheet_status'] = df_plot['in_spreadsheet'].map({True: 'In Spreadsheet', False: 'Not in Spreadsheet'})

# Calculate fractions for each combined group (for annotations)
df_plot['combined_group'] = df_plot['spreadsheet_status'] + ', ' + df_plot['institution']
group_fractions = df_plot.groupby('combined_group')['has_specimen_procedures'].agg(['count', 'sum'])
group_labels = {}
for group in group_fractions.index:
    total = group_fractions.loc[group, 'count']
    with_procedures = group_fractions.loc[group, 'sum']
    fraction = with_procedures / total if total > 0 else 0
    # Extract just the institution part for cleaner labels
    institution_part = group.split(', ')[1]  # Get the institution part after the comma
    group_labels[group] = f"{institution_part} ({with_procedures}/{total} = {fraction:.2f})"

# Use combined_group labels for the institution_annotated column
df_plot['institution_annotated'] = df_plot['combined_group'].map(group_labels)

# Add jitter to y-axis to separate overlapping points
np.random.seed(42)  # For reproducible jitter
jitter_amount = 0.20
df_plot['y_jittered'] = df_plot['has_specimen_procedures'].astype(int) + np.random.uniform(-jitter_amount, jitter_amount, len(df_plot))

# Create interactive scatter plot with TWO COLORS and unique symbols
fig = px.scatter(
    df_plot, 
    x='collection_date_dt', 
    y='y_jittered',
    color='spreadsheet_status',  # Only 2 colors: red/blue
    symbol='institution_annotated',  # Use combined_group labels with correct fractions
    hover_data=['subject_id', 'collection_date', 'asset_name', 'procedures_status', 'project_name'],
    labels={
        'collection_date_dt': 'Collection Date',
        'y_jittered': 'Has Specimen Procedures',
        'spreadsheet_status': 'Spreadsheet Status',
        'institution_annotated': 'Institution'
    },
    title='SmartSPIM Specimen Procedures Over Time',
    color_discrete_map={'In Spreadsheet': 'red', 'Not in Spreadsheet': 'blue'},
    symbol_map={
        group_labels.get('In Spreadsheet, AIND', 'In Spreadsheet, AIND'): 'circle',
        group_labels.get('Not in Spreadsheet, AIND', 'Not in Spreadsheet, AIND'): 'circle',
        group_labels.get('In Spreadsheet, AIBS', 'In Spreadsheet, AIBS'): 'diamond', 
        group_labels.get('Not in Spreadsheet, AIBS', 'Not in Spreadsheet, AIBS'): 'diamond',
        group_labels.get('In Spreadsheet, Unknown', 'In Spreadsheet, Unknown'): 'square',
        group_labels.get('Not in Spreadsheet, Unknown', 'Not in Spreadsheet, Unknown'): 'square',
        group_labels.get('Not in Spreadsheet, NYU', 'Not in Spreadsheet, NYU'): 'x',
        group_labels.get('In Spreadsheet, AI', 'In Spreadsheet, AI'): 'cross',
        group_labels.get('Not in Spreadsheet, AI', 'Not in Spreadsheet, AI'): 'cross',
        group_labels.get('Not in Spreadsheet, Columbia', 'Not in Spreadsheet, Columbia'): 'triangle-up'
    }
)

# Update layout
fig.update_layout(
    width=1500,
    height=600,
    yaxis=dict(
        tickmode='array',
        tickvals=[0, 1],
        ticktext=['False', 'True'],
        range=[-0.3, 1.3]
    )
)

# Update marker size and opacity
fig.update_traces(marker=dict(size=8, opacity=0.7))

fig.show()

print(f"Plotting {len(df_plot)} subjects with known collection dates")
print(f"Date range: {df_plot['collection_date_dt'].min()} to {df_plot['collection_date_dt'].max()}")

# Print the group fractions to verify the fix
print("\nGroup fractions (this should show different counts for each combination):")
for group, label in group_labels.items():
    print(f"  {label}")

fig.write_html("smartspim_specimen_procedures_plot.html")


Plotting 5409 subjects with known collection dates
Date range: 2022-05-31 00:00:00 to 2025-08-19 00:00:00

Group fractions (this should show different counts for each combination):
  AI (2/2 = 1.00)
  AIBS (12/25 = 0.48)
  AIND (1630/2733 = 0.60)
  Unknown (12/13 = 0.92)
  AI (0/7 = 0.00)
  AIBS (44/1756 = 0.03)
  AIND (203/613 = 0.33)
  Columbia (2/2 = 1.00)
  NYU (0/2 = 0.00)
  Unknown (35/256 = 0.14)


In [22]:
len(summary_df.query("institution == 'AIBS' and in_spreadsheet == True and has_specimen_procedures == True"))

12